In [16]:
import os
import pandas as pd
import face_recognition
import matplotlib.pyplot as plt
import cv2
import numpy as np

ruta_carpeta = '/home/waldos/Documents/2doCodigo/TopicoIA/BD/corpus_images'

In [19]:
#Crear lista para almacenar datos de imagenes
directorio = {'Imagen':[], 'Caracteristicas':[], 'Rostro':[], 'Etiqueta': []}

for nombre_carpeta in os.listdir(ruta_carpeta):
    ruta_carpeta = os.path.join(ruta_carpeta, nombre_carpeta)
    if os.path.isdir(ruta_carpeta):
        for nombre_archivo in os.listdir(ruta_carpeta):
            try:
                ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
                imagen_BGR = cv2.imread(ruta_archivo)

                imagen_RGB = cv2.cvtColor(imagen_BGR, cv2.COLOR_BGR2RGB)

                imagen_redimensionada = cv2.resize(imagen_RGB,(150,150))

                caracteristicas_faciales = face_recognition.face_landmarks(imagen_redimensionada)
                
                arriba,derecha,abajo,izquierda = face_recognition.face_locations(imagen_redimensionada)[0]

                cara_recortada = imagen_redimensionada[arriba:abajo, izquierda:derecha]

                directorio['Imagen'].append(imagen_redimensionada)
                directorio['Caracteristicas'].append(caracteristicas_faciales)
                directorio['Rostro'].append(cara_recortada)
                directorio['Etiqueta'].append(nombre_carpeta)
                

            except Exception as e:
                print("Error en archivo:"+str(nombre_archivo)+":"+str(e))

FileNotFoundError: [Errno 2] No such file or directory: '/home/waldos/Documents/2doCodigo/TopicoIA/BD/corpus_images/focused/relaxed/excited/engaged/interested/bored'